# COVID 19 API

In [44]:
import requests
import pandas as pd

# URLs to Access API End points


In [46]:
india_url = "https://api.covid19api.com/country/india/status/confirmed/live?from=2020-01-01T00:00:00Z&to=2023-01-01T00:00:00Z"

russia_url = "https://api.covid19api.com/country/russia/status/confirmed/live?from=2020-01-01T00:00:00Z&to=2023-01-01T00:00:00Z"

canada_url = "https://api.covid19api.com/country/canada/status/confirmed/live?from=2020-01-01T00:00:00Z&to=2023-01-01T00:00:00Z"


In [47]:

# sending a GET request to this URL;server responds with data in JSON format
india_response = requests.get(india_url)

#parsing the JSON data and converting it to a Python object
india_data = india_response.json()

#converting the data to a pandas dataframe 
india_df = pd.DataFrame(india_data)

#dropping columns that we are not interested in
india_df = india_df.drop(['CountryCode', 'Province', 'City', 'CityCode','Lat','Lon', 'Status'], axis=1)

#renaming columns to be more descriptive to prepare for upcoming merge
india_df = india_df.rename(columns={'Cases': 'Covid Cases in India'})

india_df


,Country,Covid Cases in India,Date
0,India,0,2020-01-22T00:00:00Z
1,India,0,2020-01-23T00:00:00Z
2,India,0,2020-01-24T00:00:00Z
3,India,0,2020-01-25T00:00:00Z
4,India,0,2020-01-26T00:00:00Z
...,...,...,...
1071,India,44679139,2022-12-28T00:00:00Z
1072,India,44679382,2022-12-29T00:00:00Z
1073,India,44679608,2022-12-30T00:00:00Z
1074,India,44679873,2022-12-31T00:00:00Z


# US df

In [48]:

#sending a GET request to this URL; server responds with data in JSON format
russia_response = requests.get(russia_url)

#parsing the JSON data and converting it to a Python object
russia_data = russia_response.json()

#converting to pandas dataframe
russia_df = pd.DataFrame(russia_data)

#dropping columns we aren't interested in
russia_df = russia_df.drop(['CountryCode', 'Province', 'City', 'CityCode','Lat','Lon', 'Status'], axis=1)

#renaming columns to be more descriptive 
russia_df = russia_df.rename(columns={'Cases': 'Covid Cases in Russia'})

russia_df

,Country,Covid Cases in Russia,Date
0,Russian Federation,0,2020-01-22T00:00:00Z
1,Russian Federation,0,2020-01-23T00:00:00Z
2,Russian Federation,0,2020-01-24T00:00:00Z
3,Russian Federation,0,2020-01-25T00:00:00Z
4,Russian Federation,0,2020-01-26T00:00:00Z
...,...,...,...
1071,Russian Federation,21473189,2022-12-28T00:00:00Z
1072,Russian Federation,21479298,2022-12-29T00:00:00Z
1073,Russian Federation,21485046,2022-12-30T00:00:00Z
1074,Russian Federation,21490515,2022-12-31T00:00:00Z


In [49]:
#sending a GET request to this URL; server responds with data in JSON format
canada_response = requests.get(canada_url)

#parsing the JSON data and converting it to a Python object
canada_data = canada_response.json()

#converting data to pandas dataframe
canada_df = pd.DataFrame(canada_data)

# dropping columns we aren't interested in
canada_df = canada_df.drop(['CountryCode', 'Province', 'City', 'CityCode','Lat','Lon', 'Status'], axis=1)

#renaming columns to be more descriptive
canada_df = canada_df.rename(columns={'Cases': 'Covid Cases in Canada'})

canada_df



,Country,Covid Cases in Canada,Date
0,Canada,0,2020-01-22T00:00:00Z
1,Canada,2,2020-01-23T00:00:00Z
2,Canada,3,2020-01-24T00:00:00Z
3,Canada,3,2020-01-25T00:00:00Z
4,Canada,3,2020-01-26T00:00:00Z
...,...,...,...
1071,Canada,3878788,2022-12-28T00:00:00Z
1072,Canada,3879896,2022-12-29T00:00:00Z
1073,Canada,3881877,2022-12-30T00:00:00Z
1074,Canada,3882688,2022-12-31T00:00:00Z


# merging the three countries

In [42]:
#Merging the DataFrames on the 'Date' column
merged_df = pd.merge(russia_df, canada_df, on='Date').merge(india_df, on='Date')

#Dropping columns that indicate country, no longer necessary in the dataframe 
merged_df = merged_df.drop(['Country_x', 'Country_y', 'Country'], axis=1)

#Moving the Date column to the far left of the dataframe
merged_df.insert(loc=0, column='Date', value=merged_df.pop('Date'))

merged_df

,Date,Covid Cases in Russia,Covid Cases in Canada,Covid Cases in India
0,2020-01-22T00:00:00Z,0,0,0
1,2020-01-23T00:00:00Z,0,2,0
2,2020-01-24T00:00:00Z,0,3,0
3,2020-01-25T00:00:00Z,0,3,0
4,2020-01-26T00:00:00Z,0,3,0
...,...,...,...,...
1071,2022-12-28T00:00:00Z,21473189,3878788,44679139
1072,2022-12-29T00:00:00Z,21479298,3879896,44679382
1073,2022-12-30T00:00:00Z,21485046,3881877,44679608
1074,2022-12-31T00:00:00Z,21490515,3882688,44679873


# new cases data frame

In [43]:
#Create a new column that is the difference between consecutive rows to find the new COVID cases
merged_df['New COVID Cases in Russia'] = merged_df['Covid Cases in Russia'].diff().fillna(0).astype(int)
merged_df['New COVID Cases in Canada'] = merged_df['Covid Cases in Canada'].diff().fillna(0).astype(int)
merged_df['New COVID Cases in India'] = merged_df['Covid Cases in India'].diff().fillna(0).astype(int)

#Dropping Covid cases columns for each country as they are no longer needed and renaming the dataframe
new_covid_cases_df = merged_df.drop(['Covid Cases in Russia', 'Covid Cases in Canada', 'Covid Cases in India'], axis=1)

new_covid_cases_df

,Date,New COVID Cases in Russia,New COVID Cases in Canada,New COVID Cases in India
0,2020-01-22T00:00:00Z,0,0,0
1,2020-01-23T00:00:00Z,0,2,0
2,2020-01-24T00:00:00Z,0,1,0
3,2020-01-25T00:00:00Z,0,0,0
4,2020-01-26T00:00:00Z,0,0,0
...,...,...,...,...
1071,2022-12-28T00:00:00Z,6139,952,268
1072,2022-12-29T00:00:00Z,6109,1108,243
1073,2022-12-30T00:00:00Z,5748,1981,226
1074,2022-12-31T00:00:00Z,5469,811,265
